# Sentiment Score for Google Maps Restaurant Reviews Around McGill

Our project implements a sentiment analysis pipeline using VADER + Custom Lexicons (Gen Z vocabulary: English + French).

Objective: to examine whether there are differences between the general population and Generation Z in their food consumption around the university, with a particular focus on value for money, defined as the balance between price and quality.


In [91]:
#%pip install vaderSentiment
# Import required libraries
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re

print("Libraries imported successfully!")

Libraries imported successfully!


### Data

Inspection, Cleaning and Preprocessing


In [92]:
# Inspect reviews dataset
reviews_sample = pd.read_csv('mcgill_restaurant_reviews.csv', nrows=2)
print("Reviews CSV Columns:")
print(reviews_sample.columns.tolist())
print(f"\nTotal columns: {len(reviews_sample.columns)}")
print("\nReview text columns found:")
review_cols = [col for col in reviews_sample.columns if col.startswith('review') and col.endswith('_text')]
print(review_cols)
print(f"\nRestaurant identifier columns: place_id, name")

Reviews CSV Columns:
['place_id', 'name', 'primary_type', 'address', 'latitude', 'longitude', 'rating', 'user_rating_count', 'distance_to_center_m', 'review1_rating', 'review1_text', 'review1_author', 'review1_publish_time', 'review2_rating', 'review2_text', 'review2_author', 'review2_publish_time', 'review3_rating', 'review3_text', 'review3_author', 'review3_publish_time', 'review4_rating', 'review4_text', 'review4_author', 'review4_publish_time', 'review5_rating', 'review5_text', 'review5_author', 'review5_publish_time']

Total columns: 29

Review text columns found:
['review1_text', 'review2_text', 'review3_text', 'review4_text', 'review5_text']

Restaurant identifier columns: place_id, name


In [68]:
# Inspect custom lexicon
lexicon_df = pd.read_csv('complete_lexicon.csv')
print("Lexicon CSV Columns:", lexicon_df.columns.tolist())
print(f"\nTotal lexicon entries: {len(lexicon_df)}")
print(f"\nScore range: {lexicon_df['Score'].min()} to {lexicon_df['Score'].max()}")
print("\nSample entries:")
print(lexicon_df.head(10))
print("\nMultiword phrases (sample):")
multiword = lexicon_df[lexicon_df['Word'].str.contains(' ', na=False)]
print(multiword.head(10) if len(multiword) > 0 else "No multiword phrases found")

Lexicon CSV Columns: ['Word', 'Score']

Total lexicon entries: 109

Score range: -5 to 5

Sample entries:
              Word  Score
0            cheap      4
1       affordable      3
2           budget      3
3  budget-friendly      4
4      inexpensive      3
5         low-cost      3
6       reasonable      2
7             fair      2
8             deal      3
9          bargain      4

Multiword phrases (sample):
                       Word  Score
66      cheap and delicious      5
67              great value      5
68        worth every penny      5
69    amazing for the price      5
70         super affordable      5
71       fire for the price      5
72   cheap but high quality      5
73  best bang for your buck      5
74       bang for your buck      5
75      insanely good value      5


In [94]:
#Define functions for data loading, lexicon processing, and sentiment scoring.
def load_and_reshape_reviews(csv_path, filter_hotels=True):
    """
    Load reviews CSV and reshape to one row per review.
    
    Args:
        csv_path: Path to the reviews CSV file
        filter_hotels: If True, exclude hotels (primary_type == 'hotel')
    """
    df = pd.read_csv(csv_path)
    
    # Filter out hotels if requested
    if filter_hotels:
        original_count = len(df)
        df = df[df['primary_type'] != 'hotel'].copy()
        filtered_count = len(df)
        print(f"Filtered out {original_count - filtered_count} hotels, keeping {filtered_count} restaurants/cafes")
    
    # Deduplicate by place_id (keep first occurrence only)
    # This ensures each place has exactly 5 reviews (one row per place)
    before_dedup = len(df)
    df = df.drop_duplicates(subset=['place_id'], keep='first').copy()
    after_dedup = len(df)
    if before_dedup > after_dedup:
        print(f"Deduplicated: removed {before_dedup - after_dedup} duplicate place entries")
    
    # Extract all review columns
    review_cols = [col for col in df.columns if col.startswith('review') and col.endswith('_text')]
    
    # Reshape: create one row per review
    reviews_list = []
    for idx, row in df.iterrows():
        place_id = row['place_id']
        name = row['name']
        rating = row.get('rating', np.nan)
        primary_type = row.get('primary_type', '')
        
        for review_col in review_cols:
            review_text = row[review_col]
            if pd.notna(review_text) and str(review_text).strip():
                # Extract review number
                review_num = review_col.replace('review', '').replace('_text', '')
                review_rating = row.get(f'review{review_num}_rating', np.nan)
                review_author = row.get(f'review{review_num}_author', np.nan)
                
                reviews_list.append({
                    'place_id': place_id,
                    'restaurant_name': name,
                    'primary_type': primary_type,
                    'restaurant_rating': rating,
                    'review_text': review_text,
                    'review_rating': review_rating,
                    'review_author': review_author,
                    'review_number': review_num
                })
    
    return pd.DataFrame(reviews_list)



In [98]:

def load_custom_lexicon(csv_path):
    """
    Load custom lexicon and return as two dicts:
    - single_words: {word: score} for single tokens
    - multiword_phrases: {phrase: score} for multiword phrases
    """
    df = pd.read_csv(csv_path)
    single_words = {}
    multiword_phrases = {}
    
    # Handle different column name possibilities
    word_col = 'Word' if 'Word' in df.columns else 'Phrase'
    
    for _, row in df.iterrows():
        word = str(row[word_col]).strip().lower()
        score = float(row['Score'])
        
        # Check if it's a multiword phrase (contains space)
        if ' ' in word:
            multiword_phrases[word] = score
        else:
            single_words[word] = score
    
    return single_words, multiword_phrases

def load_all_lexicons():
    """
    Load all lexicons (English and French) and combine them.
    Returns: (single_words, multiword_phrases) dicts with all terms combined.
    """
    # Load English lexicon
    eng_single, eng_phrases = load_custom_lexicon('complete_lexicon.csv')
    
    # Load French Montreal lexicon (combined single words and phrases)
    df_french = pd.read_excel('lexicon_french_montreal.xlsx')
    fr_single = {}
    fr_phrases = {}
    
    # Handle different column name possibilities
    word_col = 'Word' if 'Word' in df_french.columns else 'Phrase'
    
    for _, row in df_french.iterrows():
        word = str(row[word_col]).strip().lower()
        score = float(row['Score'])
        
        # Check if it's a multiword phrase (contains space)
        if ' ' in word:
            fr_phrases[word] = score
        else:
            fr_single[word] = score
    
    # Combine all lexicons
    all_single_words = {**eng_single, **fr_single}
    all_multiword_phrases = {**eng_phrases, **fr_phrases}
    
    return all_single_words, all_multiword_phrases



In [99]:
def identify_overlap(vader_analyzer, custom_single_words):
    """Identify terms that appear in both VADER and custom lexicon."""
    vader_lexicon = vader_analyzer.lexicon
    overlap = set(vader_lexicon.keys()) & set(custom_single_words.keys())
    return overlap

def create_modified_vader(custom_single_words, override=True):
    """
    Create VADER analyzer with custom lexicon integration.
    Only single words can be added to VADER's lexicon.
    """
    analyzer = SentimentIntensityAnalyzer()
    
    if override:
        # Add/override custom lexicon entries (single words only)
        for word, score in custom_single_words.items():
            # Normalize score to VADER's scale (-4 to +4)
            # Custom lexicon uses -5 to +5, so we scale proportionally
            vader_score = (score / 5.0) * 4.0
            analyzer.lexicon[word] = vader_score
    
    return analyzer



In [100]:
#Scoring functions
def clean_text(text):
    """Clean text while preserving punctuation and emojis for VADER."""
    if pd.isna(text):
        return ""
    
    text = str(text).strip()
    # Remove excessive whitespace but keep single spaces
    text = re.sub(r'\s+', ' ', text)
    return text

def score_review(analyzer, text, custom_single_words, custom_multiword_phrases):
    """
    Score a review using VADER with custom lexicon.
    Handles both single words (via VADER lexicon) and multiword phrases (via post-processing).
    """
    cleaned = clean_text(text)
    if not cleaned:
        return {
            'compound': 0.0,
            'pos': 0.0,
            'neu': 1.0,
            'neg': 0.0,
            'custom_terms_found': []
        }
    
    # Get VADER scores (includes single-word custom lexicon entries)
    scores = analyzer.polarity_scores(cleaned)
    
    # Track custom lexicon terms found and adjust for multiword phrases
    text_lower = cleaned.lower()
    custom_terms_found = []
    phrase_adjustment = 0.0
    
    # Check single words
    for term in custom_single_words.keys():
        if term in text_lower:
            custom_terms_found.append(term)
    
    # Check multiword phrases and apply adjustment
    for phrase, score in custom_multiword_phrases.items():
        if phrase in text_lower:
            custom_terms_found.append(phrase)
            # Normalize phrase score to VADER compound scale and apply weighted adjustment
            phrase_compound = (score / 5.0) * 0.3  # Scale to compound range and weight
            phrase_adjustment += phrase_compound
    
    # Apply phrase adjustment (clamp to [-1, 1] range)
    adjusted_compound = np.clip(scores['compound'] + phrase_adjustment, -1.0, 1.0)
    
    return {
        'compound': adjusted_compound,
        'pos': scores['pos'],
        'neu': scores['neu'],
        'neg': scores['neg'],
        'custom_terms_found': custom_terms_found
    }



In [101]:
# Load data

reviews_df = load_and_reshape_reviews('mcgill_restaurant_reviews.csv', filter_hotels=True)
custom_single_words, custom_multiword_phrases = load_all_lexicons()

print(f"\n Loaded {len(reviews_df)} reviews from {reviews_df['place_id'].nunique()} restaurants/cafes")
print(f" Loaded {len(custom_single_words)} single-word entries (English + French)")
print(f" Loaded {len(custom_multiword_phrases)} multiword phrases (English + French)")

# Validate: Check that each place has exactly 5 reviews
review_counts = reviews_df.groupby('place_id').size()
places_with_wrong_count = review_counts[review_counts != 5]
if len(places_with_wrong_count) > 0:
    print(f"\n  WARNING: {len(places_with_wrong_count)} places don't have exactly 5 reviews:")
    print(places_with_wrong_count.head(10))
    # Show which places have issues
    problem_places = reviews_df[reviews_df['place_id'].isin(places_with_wrong_count.index)]
    print("\nProblem places:")
    print(problem_places[['place_id', 'restaurant_name']].drop_duplicates())
else:
    print(f"\n Validation passed: All {reviews_df['place_id'].nunique()} places have exactly 5 reviews")

# Display sample
print("\nSample reviews:")
print(reviews_df[['restaurant_name', 'primary_type', 'review_text']].head(3))

Filtered out 3 hotels, keeping 105 restaurants/cafes
Deduplicated: removed 15 duplicate place entries

 Loaded 450 reviews from 90 restaurants/cafes
 Loaded 85 single-word entries (English + French)
 Loaded 46 multiword phrases (English + French)

 Validation passed: All 90 places have exactly 5 reviews

Sample reviews:
     restaurant_name         primary_type  \
0  Crusty's Downtown  american_restaurant   
1  Crusty's Downtown  american_restaurant   
2  Crusty's Downtown  american_restaurant   

                                         review_text  
0  Lucky me, I found this spot close to my hotel ...  
1  Owner was very helpful and patient. Food was s...  
2  Amazing spot with lots of seating and good dec...  


## Text Analytics Modeling Approach

In [104]:
# Create analyzer
#Initialize VADER with custom lexicon integration and identify overlaps.
#If a token/phrase appears in both VADER and custom lexicon, use the custom lexicon score
print("Creating sentiment analyzer...")
analyzer = create_modified_vader(custom_single_words, override=True)

# Identify overlap
overlap = identify_overlap(analyzer, custom_single_words)
print(f"\n Found {len(overlap)} overlapping terms between VADER and custom lexicon")
if len(overlap) > 0:
    print(f"Sample overlaps: {list(overlap)[:10]}")

Creating sentiment analyzer...

 Found 85 overlapping terms between VADER and custom lexicon
Sample overlaps: ['legit', 'flavorful', 'low-cost', 'perfect', 'inflated', 'kind', 'fresh', 'soggy', 'frais', 'pricey']


In [106]:
# Score Reviews
# Apply sentiment analysis to all reviews

# Create baseline VADER analyzer (without custom lexicon)
baseline_analyzer = SentimentIntensityAnalyzer()

# Score reviews
print("Scoring reviews...")
results = []
for idx, row in reviews_df.iterrows():
    # Custom VADER (with GenZ lexicon)
    scores = score_review(analyzer, row['review_text'], custom_single_words, custom_multiword_phrases)
    
    # Baseline VADER (standard - no custom lexicon)
    baseline_scores = baseline_analyzer.polarity_scores(clean_text(row['review_text']))
    
    results.append({
        'place_id': row['place_id'],
        'restaurant_name': row['restaurant_name'],
        'review_text': row['review_text'],
        'review_rating': row['review_rating'],
        'compound': scores['compound'],
        'positive': scores['pos'],
        'neutral': scores['neu'],
        'negative': scores['neg'],
        'compound_vader_baseline': baseline_scores['compound'],
        'custom_terms_count': len(scores['custom_terms_found']),
        'custom_terms': ', '.join(scores['custom_terms_found'][:5])  # First 5 for display
    })

results_df = pd.DataFrame(results)
print(f"\n Scored {len(results_df)} reviews")
print("\nSample results:")
print(results_df[['restaurant_name', 'compound', 'compound_vader_baseline', 'custom_terms_count']].head())

Scoring reviews...

 Scored 450 reviews

Sample results:
     restaurant_name  compound  compound_vader_baseline  custom_terms_count
0  Crusty's Downtown    0.9749                   0.9784                   2
1  Crusty's Downtown    0.9652                   0.9459                   5
2  Crusty's Downtown    0.9555                   0.9551                   2
3  Crusty's Downtown    0.9489                   0.9568                   3
4  Crusty's Downtown    0.7506                   0.7717                   1


In [108]:
#Restaurant-Level Aggregation
#Aggregate sentiment scores to restaurant level

# Restaurant-level aggregation

restaurant_agg = results_df.groupby(['place_id', 'restaurant_name']).agg({
    'compound': ['mean', 'std', 'count'],
    'compound_vader_baseline': 'mean',
    'positive': 'mean',
    'neutral': 'mean',
    'negative': 'mean',
    'custom_terms_count': 'sum',
    'review_rating': 'mean'  # Average Google Maps rating
}).reset_index()

# Flatten column names
restaurant_agg.columns = [
    'place_id', 'restaurant_name', 'mean_compound', 'std_compound', 
    'review_count', 'mean_compound_vader_baseline', 'mean_positive', 'mean_neutral', 'mean_negative', 
    'total_custom_terms', 'avg_google_rating'
]

# Convert compound score (-1 to +1) to 1-5 scale for comparison
# Linear mapping: compound -1.0 → 1.0, 0.0 → 3.0, +1.0 → 5.0
# Note: This uses custom lexicon (English GenZ + French Montreal combined)
restaurant_agg['sentiment_score_1_5_custom'] = 3 + (restaurant_agg['mean_compound'] * 2)
restaurant_agg['sentiment_score_1_5_custom'] = restaurant_agg['sentiment_score_1_5_custom'].round(2)

# Convert baseline VADER compound score to 1-5 scale
restaurant_agg['sentiment_score_1_5_vader'] = 3 + (restaurant_agg['mean_compound_vader_baseline'] * 2)
restaurant_agg['sentiment_score_1_5_vader'] = restaurant_agg['sentiment_score_1_5_vader'].round(2)

# Calculate difference between Custom sentiment score and Google rating
restaurant_agg['score_difference_custom'] = restaurant_agg['sentiment_score_1_5_custom'] - restaurant_agg['avg_google_rating']
restaurant_agg['score_difference_custom'] = restaurant_agg['score_difference_custom'].round(2)

# Calculate difference between baseline VADER and Google rating
restaurant_agg['score_difference_vader'] = restaurant_agg['sentiment_score_1_5_vader'] - restaurant_agg['avg_google_rating']
restaurant_agg['score_difference_vader'] = restaurant_agg['score_difference_vader'].round(2)

restaurant_agg = restaurant_agg.sort_values('mean_compound', ascending=False)
print(f"\n Aggregated {len(restaurant_agg)} restaurants")
print("\nTop 5 most positive restaurants (Custom Lexicon, VADER, and Google Maps ratings):")
print(restaurant_agg[['restaurant_name', 'sentiment_score_1_5_custom', 'sentiment_score_1_5_vader', 'avg_google_rating']].head())


 Aggregated 90 restaurants

Top 5 most positive restaurants (Custom Lexicon, VADER, and Google Maps ratings):
                 restaurant_name  sentiment_score_1_5_custom  \
77              Restaurant Zante                        4.96   
75  Café Olimpico - Centre-Ville                        4.96   
74              Lola Rosa Milton                        4.95   
21         LEAVES HOUSE (McGill)                        4.95   
73                           Ryu                        4.95   

    sentiment_score_1_5_vader  avg_google_rating  
77                       4.95                5.0  
75                       4.96                5.0  
74                       4.95                5.0  
21                       4.95                5.0  
73                       4.94                4.8  


In [112]:
##Summary Statistics related to results 
#Add sentiment labels
results_df['sentiment_label'] = pd.cut(
    results_df['compound'],
    bins=[-1, -0.05, 0.05, 1],
    labels=['negative', 'neutral', 'positive']
)

# 1. Distribution of Compound Scores


print("SUMMARY")

print("\n1. Distribution Scores:")
print(results_df['compound'].describe())
print(f"Percentiles: 25%={results_df['compound'].quantile(0.25):.3f}, "
      f"50%={results_df['compound'].quantile(0.50):.3f}, "
      f"75%={results_df['compound'].quantile(0.75):.3f}")

SUMMARY

1. Distribution Scores:
count    450.000000
mean       0.710920
std        0.524446
min       -1.000000
25%        0.759675
50%        0.943350
75%        0.978900
max        1.000000
Name: compound, dtype: float64
Percentiles: 25%=0.760, 50%=0.943, 75%=0.979


In [78]:
# 2. Sentiment Classification Distribution
print("\n2. Sentiment Classification (using compound thresholds):")
sentiment_dist = results_df['sentiment_label'].value_counts(normalize=True) * 100
print(sentiment_dist)
print("\nCounts:")
print(results_df['sentiment_label'].value_counts())


2. Sentiment Classification (using compound thresholds):
sentiment_label
positive    88.418708
negative    10.467706
neutral      1.113586
Name: proportion, dtype: float64

Counts:
sentiment_label
positive    397
negative     47
neutral       5
Name: count, dtype: int64


In [79]:
# 3. Custom Lexicon Usage
print("\n3. Custom Lexicon Usage:")
print(f"   Reviews with custom terms: {(results_df['custom_terms_count'] > 0).sum()} "
      f"({(results_df['custom_terms_count'] > 0).mean()*100:.1f}%)")
print(f"   Total custom terms found: {results_df['custom_terms_count'].sum()}")
print(f"   Average custom terms per review: {results_df['custom_terms_count'].mean():.2f}")


3. Custom Lexicon Usage:
   Reviews with custom terms: 412 (91.6%)
   Total custom terms found: 1353
   Average custom terms per review: 3.01


In [80]:
# 4. Top 10 Most Positive Restaurants
print("\n4. Top 10 Most Positive Restaurants (by sentiment score 1-5):")
top_positive = restaurant_agg.head(10)[['restaurant_name', 'sentiment_score_1_5_custom', 'avg_google_rating', 'score_difference_custom', 'review_count']]
print(top_positive.to_string(index=False))


4. Top 10 Most Positive Restaurants (by sentiment score 1-5):
                 restaurant_name  sentiment_score_1_5_custom  avg_google_rating  score_difference_custom  review_count
                Restaurant Zante                        4.96                5.0                    -0.04             5
    Café Olimpico - Centre-Ville                        4.96                5.0                    -0.04             5
                Lola Rosa Milton                        4.95                5.0                    -0.05             5
           LEAVES HOUSE (McGill)                        4.95                5.0                    -0.05             5
                             Ryu                        4.95                4.8                     0.15             5
   Café Humble Lion (Sherbrooke)                        4.95                4.4                     0.55             5
          Le Poké Station McGill                        4.95                5.0                    -0.05

In [81]:
# 5. Bottom 10 Most Negative Restaurants
print("\n5. Bottom 10 Most Negative Restaurants (by sentiment score 1-5):")
bottom_negative = restaurant_agg.tail(10)[['restaurant_name', 'sentiment_score_1_5_custom', 'avg_google_rating', 'score_difference_custom', 'review_count']]
print(bottom_negative.to_string(index=False))


5. Bottom 10 Most Negative Restaurants (by sentiment score 1-5):
   restaurant_name  sentiment_score_1_5_custom  avg_google_rating  score_difference_custom  review_count
        La Cantina                        3.36                3.2                     0.16             5
        McDonald's                        3.30                2.8                     0.50             5
            Subway                        3.13                2.6                     0.53             5
Columbus Café & Co                        3.11                3.2                    -0.09             5
       Tim Hortons                        3.07                2.8                     0.27             5
          Cultures                        2.98                2.4                     0.58             5
 Second Cup Coffee                        2.95                3.2                    -0.25             5
         Starbucks                        2.64                2.0                     0.64    

In [113]:
#Sentiment Score vs Google Maps Rating Comparison

# Comparison statistics

print("SENTIMENT SCORE vs GOOGLE MAPS RATING COMPARISON")


print(f"\nSentiment Score (1-5) Statistics:")
print(restaurant_agg['sentiment_score_1_5_custom'].describe())

print(f"\nGoogle Maps Rating Statistics:")
print(restaurant_agg['avg_google_rating'].describe())

print(f"\nScore Difference Statistics (Sentiment - Google):")
print(restaurant_agg['score_difference_custom'].describe())
print(f"\n   Positive difference = Sentiment score is higher than Google rating")
print(f"   Negative difference = Sentiment score is lower than Google rating")

# Correlation
correlation = restaurant_agg['sentiment_score_1_5_custom'].corr(restaurant_agg['avg_google_rating'])
print(f"\nCorrelation between Sentiment Score and Google Rating: {correlation:.3f}")

# Restaurants with largest differences
print("\nRestaurants where Sentiment Score is much HIGHER than Google Rating:")
high_diff = restaurant_agg.nlargest(10, 'score_difference_custom')[['restaurant_name', 'sentiment_score_1_5_custom', 'avg_google_rating', 'score_difference_custom']]
print(high_diff.to_string(index=False))

print("\nRestaurants where Sentiment Score is much LOWER than Google Rating:")
low_diff = restaurant_agg.nsmallest(10, 'score_difference_custom')[['restaurant_name', 'sentiment_score_1_5_custom', 'avg_google_rating', 'score_difference_custom']]
print(low_diff.to_string(index=False))

SENTIMENT SCORE vs GOOGLE MAPS RATING COMPARISON

Sentiment Score (1-5) Statistics:
count    90.000000
mean      4.421333
std       0.655084
min       2.120000
25%       4.202500
50%       4.685000
75%       4.907500
max       4.960000
Name: sentiment_score_1_5_custom, dtype: float64

Google Maps Rating Statistics:
count    90.000000
mean      4.224444
std       0.801026
min       1.000000
25%       4.000000
50%       4.400000
75%       4.800000
max       5.000000
Name: avg_google_rating, dtype: float64

Score Difference Statistics (Sentiment - Google):
count    90.000000
mean      0.196889
std       0.351068
min      -0.600000
25%      -0.057500
50%       0.125000
75%       0.410000
max       1.150000
Name: score_difference_custom, dtype: float64

   Positive difference = Sentiment score is higher than Google rating
   Negative difference = Sentiment score is lower than Google rating

Correlation between Sentiment Score and Google Rating: 0.903

Restaurants where Sentiment Score is mu

In [115]:
#Validation: Custom Lexicon Impact
#Compare results with baseline VADER to show where custom lexicon changes the outcome.

# Create baseline VADER (without custom lexicon)
baseline_analyzer = SentimentIntensityAnalyzer()

# Find reviews where custom lexicon changes the result

print("VALIDATION: Custom Lexicon Impact")


comparison_results = []
for idx, row in results_df.iterrows():
    baseline_scores = baseline_analyzer.polarity_scores(clean_text(row['review_text']))
    baseline_compound = baseline_scores['compound']
    custom_compound = row['compound']
    
    # Check if sentiment label changed
    baseline_label = 'negative' if baseline_compound < -0.05 else ('positive' if baseline_compound > 0.05 else 'neutral')
    custom_label = row['sentiment_label']
    
    # Include if label changed OR significant difference OR custom terms were found
    if baseline_label != custom_label or abs(baseline_compound - custom_compound) > 0.1 or row['custom_terms_count'] > 0:
        comparison_results.append({
            'review_text': row['review_text'][:200] + '...' if len(row['review_text']) > 200 else row['review_text'],
            'baseline_compound': baseline_compound,
            'custom_compound': custom_compound,
            'baseline_label': baseline_label,
            'custom_label': custom_label,
            'difference': custom_compound - baseline_compound,
            'custom_terms': row['custom_terms']
        })

if comparison_results:
    comparison_df = pd.DataFrame(comparison_results)
    comparison_df = comparison_df.sort_values('difference', key=abs, ascending=False)
    
    print(f"\nFound {len(comparison_df)} reviews where custom lexicon had an impact")
    print("\nExample reviews where custom lexicon changed the result:")
    
    for i, row in comparison_df.head(3).iterrows():
        print(f"\nExample {list(comparison_df.index).index(i) + 1}:")
        print(f"Review: {row['review_text']}")
        print(f"Baseline VADER: {row['baseline_compound']:.3f} ({row['baseline_label']})")
        print(f"With Custom Lexicon: {row['custom_compound']:.3f} ({row['custom_label']})")
        print(f"Difference: {row['difference']:+.3f}")
        print(f"Custom terms found: {row['custom_terms']}")
        
else:
    print("\nNo significant differences found between baseline and custom lexicon.")

VALIDATION: Custom Lexicon Impact

Found 412 reviews where custom lexicon had an impact

Example reviews where custom lexicon changed the result:

Example 1:
Review: It's expensive for third-quintile coffee: latte wasn't very tasty and mocha was a bit too sweet. Not bad by any means, but not different than what you would get down the street. However, $18 for two d...
Baseline VADER: 0.463 (positive)
With Custom Lexicon: -0.691 (negative)
Difference: -1.155
Custom terms found: expensive, steep, tasty, bad

Example 2:
Review: Location is good and they provide free parking, which is really great for a downtown location however it is very confusing to get to the restaurant from the parking lot because you’re going through a ...
Baseline VADER: -0.123 (negative)
With Custom Lexicon: -0.786 (negative)
Difference: -0.664
Custom terms found: great, good, cold

Example 3:
Review: The service was fast and efficient. Attentive and present

We generally liked everything but nothing was wow. Fresh 

In [117]:
#Identify potential issues with the results


print("SUSPICIOUS OUTCOME CHECKS")


neutral_pct = (results_df['sentiment_label'] == 'neutral').mean() * 100
if neutral_pct > 70:
    print(f"\n  WARNING: {neutral_pct:.1f}% of reviews are neutral. This may indicate:")
    print("   - Reviews are too short or lack sentiment-bearing words")
    print("   - Lexicon mismatch (terms not appearing in reviews)")
    print("   - Suggestion: Check if custom lexicon terms are actually present in reviews")
else:
    print(f"\nNeutral reviews: {neutral_pct:.1f}% (acceptable)")

if results_df['compound'].std() < 0.1:
    print(f"\n  WARNING: Very low variance in compound scores (std={results_df['compound'].std():.3f})")
    print("   - Suggestion: Check if scoring is working correctly")
else:
    print(f"\nCompound score std: {results_df['compound'].std():.3f} (acceptable)")

custom_usage_pct = (results_df['custom_terms_count'] > 0).mean() * 100
if custom_usage_pct < 0.1:
    print(f"\n  WARNING: Only {custom_usage_pct:.1f}% of reviews contain custom lexicon terms")
    print("   - Suggestion: Verify lexicon terms match review vocabulary")
else:
    print(f"\nCustom lexicon usage: {custom_usage_pct:.1f}% of reviews contain custom terms")

SUSPICIOUS OUTCOME CHECKS

Neutral reviews: 1.1% (acceptable)

Compound score std: 0.524 (acceptable)

Custom lexicon usage: 91.6% of reviews contain custom terms


In [120]:
#Save Outputs


# Create comprehensive comparison summary

print("CREATING COMPREHENSIVE COMPARISON SUMMARY")


# Calculate difference between Custom and VADER
restaurant_agg['custom_vs_vader_difference'] = restaurant_agg['sentiment_score_1_5_custom'] - restaurant_agg['sentiment_score_1_5_vader']
restaurant_agg['custom_vs_vader_difference'] = restaurant_agg['custom_vs_vader_difference'].round(2)

# Create summary dataframe with all key metrics
summary_comparison = restaurant_agg[[
    'restaurant_name',
    'review_count',
    'avg_google_rating',
    'sentiment_score_1_5_custom',
    'sentiment_score_1_5_vader',
    'score_difference_custom',
    'score_difference_vader',
    'custom_vs_vader_difference'
]].copy()

# Rename columns for clarity
summary_comparison.columns = [
    'restaurant_name',
    'review_count',
    'google_avg_rating',
    'custom_sentiment_score',
    'vader_sentiment_score',
    'custom_vs_google_diff',
    'vader_vs_google_diff',
    'custom_vs_vader_diff'
]

print("\n Summary comparison created with the following structure:")
print(summary_comparison.head(10))

print("\n\nSummary Statistics:")
print("\nGoogle Rating:")
print(f"  Mean: {summary_comparison['google_avg_rating'].mean():.2f}")
print(f"  Std:  {summary_comparison['google_avg_rating'].std():.2f}")

print("\nCustom Lexicon Sentiment Score (1-5 scale):")
print(f"  Mean: {summary_comparison['custom_sentiment_score'].mean():.2f}")
print(f"  Std:  {summary_comparison['custom_sentiment_score'].std():.2f}")

print("\nVADER Sentiment Score (1-5 scale):")
print(f"  Mean: {summary_comparison['vader_sentiment_score'].mean():.2f}")
print(f"  Std:  {summary_comparison['vader_sentiment_score'].std():.2f}")

print("\nDifferences:")
print(f"  Custom vs Google - Mean: {summary_comparison['custom_vs_google_diff'].mean():+.2f}, Std: {summary_comparison['custom_vs_google_diff'].std():.2f}")
print(f"  VADER vs Google - Mean: {summary_comparison['vader_vs_google_diff'].mean():+.2f}, Std: {summary_comparison['vader_vs_google_diff'].std():.2f}")
print(f"  Custom vs VADER - Mean: {summary_comparison['custom_vs_vader_diff'].mean():+.2f}, Std: {summary_comparison['custom_vs_vader_diff'].std():.2f}")


CREATING COMPREHENSIVE COMPARISON SUMMARY

 Summary comparison created with the following structure:
                     restaurant_name  review_count  google_avg_rating  \
77                  Restaurant Zante             5                5.0   
75      Café Olimpico - Centre-Ville             5                5.0   
74                  Lola Rosa Milton             5                5.0   
21             LEAVES HOUSE (McGill)             5                5.0   
73                               Ryu             5                4.8   
50     Café Humble Lion (Sherbrooke)             5                4.4   
8             Le Poké Station McGill             5                5.0   
31                       Sushi Inbox             5                5.0   
54  Pizzéria NO.900 - Peel, Montréal             5                4.8   
39                      Poulet Rouge             5                5.0   

    custom_sentiment_score  vader_sentiment_score  custom_vs_google_diff  \
77                 

In [121]:
#Export results to CSV files.

print("To download the summary of all the work completed, please remove the # symbols. The document descriptions are as follows:")

print("  - review_sentiment_scores.csv: Review-level scores")
print("  - restaurant_sentiment_aggregated.csv: Restaurant-level aggregation")

if comparison_results:
    print("  - lexicon_comparison_examples.csv: Validation examples")
print("The only file to download by default is the summary of all the ratings.")
    
# Save review-level results
#results_df.to_csv('review_sentiment_scores.csv', index=False)
#print("Saved review-level results to review_sentiment_scores.csv")

# Save restaurant-level aggregation
#restaurant_agg.to_csv('restaurant_sentiment_aggregated.csv', index=False)
#print("Saved restaurant-level results to restaurant_sentiment_aggregated.csv")

# Save comprehensive summary comparison
summary_comparison.to_csv('summary_comparison_all_ratings.csv', index=False)
print("Saved comprehensive summary to summary_comparison_all_ratings.csv")

# Save validation examples if available
#if comparison_results:
 #   comparison_df.to_csv('lexicon_comparison_examples.csv', index=False)
 #   print("✓ Saved validation examples to lexicon_comparison_examples.csv")





To download the summary of all the work completed, please remove the # symbols. The document descriptions are as follows:
  - review_sentiment_scores.csv: Review-level scores
  - restaurant_sentiment_aggregated.csv: Restaurant-level aggregation
  - lexicon_comparison_examples.csv: Validation examples
The only file to download by default is the summary of all the ratings.
Saved comprehensive summary to summary_comparison_all_ratings.csv


## Additional - Rank Correlation Analysis

Rank correlation measures whether two variables have a monotonic relationship (if one increases, the other tends to consistently increase or decrease)

In [124]:
from scipy.stats import spearmanr


print("RANK CORRELATION ANALYSIS (SPEARMAN)")



# GenZ vs Google Rating
spearman_custom_google, p_value_custom_google = spearmanr(
    summary_comparison['custom_sentiment_score'],
    summary_comparison['google_avg_rating']
)
print(f"\nCustom Lexicon Sentiment Score ↔ Google Rating:")
print(f"  ρ (rho) = {spearman_custom_google:.4f}")
print(f"  p-value = {p_value_custom_google:.6f}")
print(f"  Interpretation: {'SIGNIFICANT' if p_value_custom_google < 0.05 else 'NOT significant'} (α=0.05)")

# VADER vs Google Rating
spearman_vader_google, p_value_vader_google = spearmanr(
    summary_comparison['vader_sentiment_score'],
    summary_comparison['google_avg_rating']
)
print(f"\nVADER Sentiment Score ↔ Google Rating:")
print(f"  ρ (rho) = {spearman_vader_google:.4f}")
print(f"  p-value = {p_value_vader_google:.6f}")
print(f"  Interpretation: {'SIGNIFICANT' if p_value_vader_google < 0.05 else 'NOT significant'} (α=0.05)")

# GenZ vs VADER
spearman_custom_vader, p_value_custom_vader = spearmanr(
    summary_comparison['custom_sentiment_score'],
    summary_comparison['vader_sentiment_score']
)
print(f"\nCustom Lexicon vs VADER Sentiment Scores:")
print(f"  ρ (rho) = {spearman_custom_vader:.4f}")
print(f"  p-value = {p_value_custom_vader:.6f}")
print(f"  Interpretation: {'SIGNIFICANT' if p_value_custom_vader < 0.05 else 'NOT significant'} (α=0.05)")

# Summary table of Spearman correlations
print("\n\nSUMMARY TABLE: SPEARMAN'S RANK CORRELATION")

correlation_summary = pd.DataFrame({
    'Comparison': [
        'Custom ↔ Google',
        'VADER ↔ Google', 
        'Custom ↔ VADER'
    ],
    'Spearman ρ': [spearman_custom_google, spearman_vader_google, spearman_custom_vader],
    'p-value': [p_value_custom_google, p_value_vader_google, p_value_custom_vader],
    'Significance': [
        'YES' if p_value_custom_google < 0.05 else 'NO',
        'YES' if p_value_vader_google < 0.05 else 'NO',
        'YES' if p_value_custom_vader < 0.05 else 'NO'
    ]
})

print(correlation_summary.to_string(index=False))




RANK CORRELATION ANALYSIS (SPEARMAN)

Custom Lexicon Sentiment Score ↔ Google Rating:
  ρ (rho) = 0.7945
  p-value = 0.000000
  Interpretation: SIGNIFICANT (α=0.05)

VADER Sentiment Score ↔ Google Rating:
  ρ (rho) = 0.7473
  p-value = 0.000000
  Interpretation: SIGNIFICANT (α=0.05)

Custom Lexicon vs VADER Sentiment Scores:
  ρ (rho) = 0.9693
  p-value = 0.000000
  Interpretation: SIGNIFICANT (α=0.05)


SUMMARY TABLE: SPEARMAN'S RANK CORRELATION
     Comparison  Spearman ρ      p-value Significance
Custom ↔ Google    0.794467 9.237328e-21          YES
 VADER ↔ Google    0.747342 2.674896e-17          YES
 Custom ↔ VADER    0.969298 2.122714e-55          YES


The Custom GenZ Lexicon and VADER methods show very strong ranking consistency (ρ=0.9693, p=0.000000). Despite using different vocabularies, both methods rank restaurants almost identically, suggesting that standard VADER already captures Gen Z sentiment reasonably well for this dataset.
strongly aligns with Google ratings (ρ=0.7945, p=0.000000). Text-based sentiment extraction via the custom lexicon validates the star ratings reviewers provide, suggesting review text and numeric ratings capture similar underlying sentiment.

Incorporating Gen Z slang and Montreal French expressions helped slightly.